In [2]:
from common import *
from experiment import *
from data import RawData, DirectRawData, PlottableData
from typing import Union, List
from itertools import chain
from heuristic import *
import numpy as np

# Splittability heuristic model

In [3]:
# class NetworkModel:
#     def __init__(self, delay: int, loss: Union[int, float, str], bw: int):
#         self.delay = delay
#         self.loss = loss
#         self.bw = bw

# def compose(s1: NetworkModel, s2: NetworkModel) -> NetworkModel:
#     delay = s1.delay + s2.delay
#     if s1.delay == 1 or s2.delay == 1:
#         delay -= 1
#     loss = s1.loss + s2.loss
#     bw = min(s1.bw, s2.bw)
#     return NetworkModel(delay, loss, bw)

In [4]:
# def get_data_size(bottleneck_bw):
#     return int(10*1000000*bottleneck_bw/8)  # 10x the bottleneck bandwidth

# class TreatmentData:
#     def __init__(
#         self,
#         tcp_treatment: Treatment,
#         pep_treatment: Treatment,
#         data: PlottableData,
#         direct_data: PlottableData,
#     ):
#         self.tcp = tcp_treatment.label()
#         self.pep = pep_treatment.label()
#         self.data = data
#         self.direct_data = direct_data
    
#     def goodput(self, s) -> Optional[float]:
#         ns = DirectNetworkSetting(delay=s.delay, loss=s.loss, bw=s.bw)
#         data_size = get_data_size(s.bw)
#         result = self.direct_data.data[self.tcp][ns.label()].get(data_size)
#         if result is None:
#             return None
#         else:
#             return result.p(50)
    
#     def pred_split_goodput(self, s1: NetworkModel, s2: NetworkModel) -> Optional[float]:
#         goodput1 = self.goodput(s1)
#         goodput2 = self.goodput(s2)
#         if goodput1 is None or goodput2 is None:
#             return None
#         return min(goodput1, goodput2)
    
#     def pred_e2e_goodput(self, s1: NetworkModel, s2: NetworkModel) -> Optional[float]:
#         s = compose(s1, s2)
#         return self.goodput(s)
    
#     def real_split_goodput(self, ns: NetworkSetting) -> Optional[float]:
#         data_size = get_data_size(min(ns.get('bw1'), ns.get('bw2')))
#         goodput = self.data.data[self.pep][ns.label()].get(data_size)
#         return None if goodput is None else goodput.p(50)
    
#     def real_e2e_goodput(self, ns: NetworkSetting) -> Optional[float]:
#         data_size = get_data_size(min(ns.get('bw1'), ns.get('bw2')))
#         goodput = self.data.data[self.tcp][ns.label()].get(data_size)
#         return None if goodput is None else goodput.p(50)

# Plotting function

In [5]:
def plot_throughput_accuracy(
    treatment: TreatmentData,
    x_labels: List,
    y_labels: List,
    network_settings: List[NetworkSetting],
    title: str,
    split: bool,
    xlabel: str,
    ylabel: str,
    accuracy_vmin: float=-1.0,
    accuracy_vmax: float=1.0,
):
    """
    - treatment: Contains the TCP and PEP treatment for a congestion control algorithm.
    - x_labels: The first axis on which to vary the network settings.
    - y_labels: The second axis on which to vary the network settings.
    - network_settings: A len(y_labels) x len(x_labels) array of network settings generated from the x_labels and y_labels.
    - title: The accuracy metric being assessed, the title of the plot.
    - split: If True, evaluates split accuracy. Otherwise end-to-end accuracy.
    """
    
    # Generate data for the three heatmaps from the network settings
    def init_data():
        return [[0 for _ in range(len(x_labels))] for _ in range(len(y_labels))]
    data_real = init_data()
    data_pred = init_data()
    data_accuracy = init_data()
    for i in range(len(y_labels)):
        for j in range(len(x_labels)):
            ns = network_settings[i][j]
            bottleneck_bw = min(ns.get('bw1'), ns.get('bw2'))
            s1 = NetworkModel(delay=ns.get('delay1'), bw=ns.get('bw1'), loss=ns.get('loss1'))
            s2 = NetworkModel(delay=ns.get('delay2'), bw=ns.get('bw2'), loss=ns.get('loss2'))
            if split:
                real = treatment.real_split_goodput(ns)
                pred = treatment.pred_split_goodput(s1, s2)
            else:
                real = treatment.real_e2e_goodput(ns)
                pred = treatment.pred_e2e_goodput(s1, s2)

            # Set the values if they are not None
            if real is not None:
                data_real[i][j] = real / bottleneck_bw
            if pred is not None:
                data_pred[i][j] = pred / bottleneck_bw
            if real is not None and pred is not None:
                accuracy = (pred - real) / real
                data_accuracy[i][j] = accuracy

    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
    
    # Function to plot a single heatmap
    from matplotlib.colors import TwoSlopeNorm
    def plot_heatmap(ax, data, title, accuracy=False):
        if accuracy:
            im = ax.imshow(data, cmap='coolwarm', interpolation='nearest',
                           norm=TwoSlopeNorm(vmin=accuracy_vmin, vcenter=0, vmax=accuracy_vmax))
        else:
            im = ax.imshow(data, cmap='plasma', interpolation='nearest', vmin=0, vmax=1)
        ax.set_xticks(np.arange(len(x_labels)))
        ax.set_xticklabels(x_labels, rotation=45, ha='right')
        ax.set_yticks(np.arange(len(y_labels)))
        ax.set_yticklabels(y_labels)
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        
        # Add numerical labels to each square
        for i in range(len(y_labels)):
            for j in range(len(x_labels)):
                ax.text(j, i, f"{data[i][j]:.2f}", ha='center', va='center',
                        color='white' if data[i][j] < 0.5 else 'black')
        
        return im
    
    # Plot each heatmap
    im1 = plot_heatmap(axes[0], data_real, f'{treatment.tcp} {title} (Real)')
    im2 = plot_heatmap(axes[1], data_pred, f'{treatment.tcp} {title} (Predicted)')
    im3 = plot_heatmap(axes[2], data_accuracy, f'{treatment.tcp} {title} (Accuracy)', accuracy=True)
    
    # Add colorbars
    fig.colorbar(im1, ax=axes[0], orientation='vertical', shrink=0.65, label='% Bottleneck Bandwidth Utilization')
    fig.colorbar(im2, ax=axes[1], orientation='vertical', shrink=0.65, label='% Bottleneck Bandwidth Utilization')
    fig.colorbar(im3, ax=axes[2], orientation='vertical', shrink=0.65, label='Accuracy')
    
    # Show the plot
    plt.show()

# Data generation functions

In [11]:
def _tuple_list_to_ordered_set(tuple_list):
    arr = list(chain.from_iterable(tuple_list))
    arr = list(set(arr))
    arr.sort()
    return arr

def gen_direct_data_from_splits(treatments, bw_splits, delay_splits, loss_splits, timeout=180, num_trials=1):
    plottable_data = gen_direct_data(
        losses=_tuple_list_to_ordered_set(loss_splits),
        delays=_tuple_list_to_ordered_set(delay_splits),
        bws=_tuple_list_to_ordered_set(bw_splits),
        treatments=treatments,
        num_trials=num_trials,
        timeout=timeout,
    )
    return plottable_data

def gen_data(treatments, network_settings, timeout=None, execute=False, num_trials=1):
    network_settings = list(chain.from_iterable(network_settings))
    data_sizes = [get_data_size(min(ns.get('bw1'), ns.get('bw2'))) for ns in network_settings]
    exp = Experiment(
        num_trials, treatments, network_settings, data_sizes,
        timeout=timeout,
        cartesian=False,
    )
    raw_data = RawData(exp, execute=execute)
    plottable_data = PlottableData(raw_data, metric='throughput_mbps')
    return plottable_data

In [7]:
def gen_network_settings_without_loss(bw_splits, delay_splits):
    network_settings = []
    for bw1, bw2 in bw_splits:
        nses = []
        for delay1, delay2 in delay_splits:
            ns = NetworkSetting(delay1=delay1, delay2=delay2, bw1=bw1, bw2=bw2, loss1=0, loss2=0)
            nses.append(ns)
        network_settings.append(nses)
    return network_settings

In [8]:
def gen_x_labels_with_loss(bw_splits, delay_splits):
    labels = []
    for bw1, bw2 in bw_splits:
        for delay1, delay2 in delay_splits:
            label = f'{bw1}_{bw2}_{delay1}_{delay2}'
            labels.append(label)
    return labels

def gen_network_settings_with_loss(bw_splits, delay_splits, loss_splits):
    network_settings = []
    for loss1, loss2 in loss_splits:
        nses = []
        for bw1, bw2 in bw_splits:
            for delay1, delay2 in delay_splits:
                ns = NetworkSetting(delay1=delay1, delay2=delay2, bw1=bw1, bw2=bw2, loss1=loss1, loss2=loss2)
                nses.append(ns)
        network_settings.append(nses)
    return network_settings

# Throughput accuracy

## Without loss

In [13]:
def plot_throughput_accuracy_without_loss(tcp_treatment, pep_treatment,
                                          e2e=False, split=False, execute=False, num_trials=10):
    # Define parameters
    delay_splits = [(1, 80), (20, 60), (40, 40), (60, 20), (80, 1)]
    bw_splits = [(10, 50), (10, 30), (10, 10), (30, 10), (50, 10)]
    network_settings = gen_network_settings_without_loss(bw_splits, delay_splits)
    
    # Generate data
    data = gen_data([tcp_treatment, pep_treatment], network_settings, timeout=180,
                    execute=execute, num_trials=num_trials)
    direct_data = gen_direct_data_from_splits([tcp_treatment], bw_splits=bw_splits, delay_splits=delay_splits,
                                  loss_splits=[(0, 0)], timeout=None, num_trials=num_trials)
    treatment = TreatmentData(tcp_treatment, pep_treatment=pep_treatment, onehop_data=data, direct_data=direct_data)
    
    # Plot heatmaps
    xlabel = 'Delay Split (ms)'
    ylabel = 'Bandwidth Split (Mbit/s)'
    if e2e:
        plot_throughput_accuracy(treatment, x_labels=delay_splits, y_labels=bw_splits,
                                 network_settings=network_settings, xlabel=xlabel, ylabel=ylabel,
                                 title='End-to-End Throughput,\nWithout Loss', split=False)
    if split:
        plot_throughput_accuracy(treatment, x_labels=delay_splits, y_labels=bw_splits,
                                 network_settings=network_settings, xlabel=xlabel, ylabel=ylabel,
                                 title='Split Throughput,\nWithout Loss', split=True)

In [ ]:
plot_throughput_accuracy_without_loss(TCP_BBRV3, PEP_BBRV3, e2e=True, num_trials=1, execute=True)
plot_throughput_accuracy_without_loss(TCP_BBRV3, PEP_BBRV3, split=True, num_trials=1, execute=True)

50
13.439502239227295ulation/main.py --timeout 180 --bw1 10 --bw2 50 --delay2 80 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
12.702264785766602ulation/main.py --timeout 180 --bw1 10 --bw2 50 --delay1 20 --delay2 60 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
12.865572214126587ulation/main.py --timeout 180 --bw1 10 --bw2 50 --delay1 40 --delay2 40 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
23.515873670578003ulation/main.py --timeout 180 --bw1 10 --bw2 50 --delay1 60 --delay2 20 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
12.624822616577148ulation/main.py --timeout 180 --bw1 10 --bw2 50 --delay1 80 --delay2 1 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
12.715826272964478ulation/main.py --timeout 180 --bw1 10 --bw2 30 --delay2 80 --loss1 0 --loss2 0 -t 1 --label tcp_bbr3 tcp -cca bbr -n 12500000 
23.470786094665527ulation/main.py --timeout 180 --bw1 10 --bw2 30 --delay1

## With loss

In [22]:
def plot_throughput_accuracy_with_loss(tcp_treatment, pep_treatment,
                                       e2e=False, split=False, execute=False, num_trials=1):
    # Define parameters
    delay_splits = [(1, 80), (40, 40), (80, 1)]
    bw_splits = [(10, 50), (10, 10), (50, 10)]
    loss_splits = [(4, 0), (3, 1), (2, 2), (1, 3), (0, 4)]
    network_settings = gen_network_settings_with_loss(bw_splits, delay_splits, loss_splits)
    
    # Generate data
    data = gen_data([tcp_treatment, pep_treatment], network_settings, timeout=180, execute=execute, num_trials=num_trials)
    direct_data = gen_direct_data_from_splits([tcp_treatment], bw_splits=bw_splits, delay_splits=delay_splits,
                                              loss_splits=loss_splits, timeout=None, num_trials=num_trials)
    treatment = TreatmentData(tcp_treatment, pep_treatment=pep_treatment, onehop_data=data, direct_data=direct_data)
    
    # Plot heatmaps
    x_labels = gen_x_labels_with_loss(bw_splits, delay_splits)
    xlabel = 'Bandwidth (Mbit/s) and Delay (ms) Split'
    ylabel = 'Loss Split (%)'
    if e2e:
        plot_throughput_accuracy(treatment, x_labels=x_labels, y_labels=loss_splits, network_settings=network_settings, title='End-to-End Throughput,\nWith Loss', split=False, xlabel=xlabel, ylabel=ylabel)
    if split:
        plot_throughput_accuracy(treatment, x_labels=x_labels, y_labels=loss_splits, network_settings=network_settings, title='Split Throughput,\nWith Loss', split=True, xlabel=xlabel, ylabel=ylabel)

In [ ]:
plot_throughput_accuracy_with_loss(TCP_BBRV3, PEP_BBRV3, e2e=True, num_trials=1, execute=True)
plot_throughput_accuracy_with_loss(TCP_BBRV3, PEP_BBRV3, split=True, num_trials=1, execute=True)